In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
digits = load_digits()
x,y = digits.data, digits.target

# Normalizing
scaler = StandardScaler()
scaler.fit_transform(x)

#convert labels to onehot encoding
def one_hot_encode(labels, num_classes=10):
    return np.eye(num_classes)[labels]

y=one_hot_encode(y)
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=42)

# Define Neural Network

In [12]:
class NeuralNetwork:
    def __init__(self,input_size,hidden_size,output_size):
        # initializing weights and biases
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        # Forward propagation
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y, output, learning_rate=0.01):
        # Backward propagation
        self.error = y - output
        self.delta2 = self.error * self.sigmoid_derivative(output)
        self.error_hidden = np.dot(self.delta2, self.W2.T)
        self.delta1 = self.error_hidden * self.sigmoid_derivative(self.a1)
        # Update weights and biases
        self.W2 += learning_rate * np.dot(self.a1.T, self.delta2)
        self.b2 += learning_rate * np.sum(self.delta2, axis=0, keepdims=True)
        self.W1 += learning_rate * np.dot(X.T, self.delta1)
        self.b1 += learning_rate * np.sum(self.delta1, axis=0, keepdims=True)
    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Train and test

In [14]:
# Initialize and train
nn = NeuralNetwork(input_size=64, hidden_size=16, output_size=10)
nn.train(xtrain, ytrain, epochs=1000)
# Test accuracy
def predict(X):
    return np.argmax(nn.forward(X), axis=1)
    
ypred = predict(xtest)
ytrue = np.argmax(ytest, axis=1)
accuracy = np.mean(ypred == ytrue)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 0, Loss: 0.2524
Epoch 100, Loss: 0.0900
Epoch 200, Loss: 0.0900
Epoch 300, Loss: 0.0900
Epoch 400, Loss: 0.0900
Epoch 500, Loss: 0.0900
Epoch 600, Loss: 0.0900
Epoch 700, Loss: 0.0900
Epoch 800, Loss: 0.0900
Epoch 900, Loss: 0.0900
Test Accuracy: 0.0778
